In [1]:
import tensorflow as tf
from ParticleNet import get_particle_net
import yaml

gpu = True
if not gpu:
# Establecer para que TensorFlow solo use CPU
  tf.config.set_visible_devices([], 'GPU')

# Verificar si solo se está usando CPU
  print("Dispositivos físicos disponibles:", tf.config.list_physical_devices('GPU'))

# Tu código de TensorFlow aquí
!export LD_LIBRARY_PATH=/usr/local/cuda/lib64:$LD_LIBRARY_PATH

# !export LD_LIBRARY_PATH=/mnt/c/Users/olmic/Desktop/Personal/Máster/TFM/Código/TFM-CTLearn/cudnn/cudnn-linux-x86_64-8.9.2.26_cuda11-archive/lib:$LD_LIBRARY_PATH
# !export CUDNN_INCLUDE_DIR=/mnt/c/Users/olmic/Desktop/Personal/Máster/TFM/Código/TFM-CTLearn/cudnn/cudnn-linux-x86_64-8.9.2.26_cuda11-archive/include


2024-07-11 20:14:10.107550: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-11 20:14:10.168350: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-11 20:14:10.168391: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-11 20:14:10.168415: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-11 20:14:10.179440: I tensorflow/core/platform/cpu_feature_g

In [2]:
with open('PN_config.yml', 'r') as myconfig:
  config = yaml.load(myconfig, Loader=yaml.Loader)
model_config = config["Model Parameters"]

In [3]:
# Generación de datos
from DataLoader import PCDataGenerator
import h5py
import numpy as np

data_path = "../data/dataset_500_rel_coords.h5"
# Dividir el dataset en train, test y validation
with h5py.File(data_path, 'r') as f:
    all_indices = list(f.keys())

# Suponiendo que quieres una división 70% train, 15% validation, 15% test
np.random.shuffle(all_indices)
train_split = int(0.7 * len(all_indices))
val_split = int(0.85 * len(all_indices))

train_indices = all_indices[:train_split]
val_indices = all_indices[train_split:val_split]
test_indices = all_indices[val_split:]

# Crear generadores para cada partición
batch_size = 16
train_data = PCDataGenerator(data_path, train_indices, batch_size=batch_size, shuffle=True)
val_data = PCDataGenerator(data_path, val_indices, batch_size=batch_size, shuffle=True)
test_data = PCDataGenerator(data_path, test_indices, batch_size=batch_size, shuffle=False)

In [4]:
model_config["model_directory"] = "./"

In [5]:
model, inputs = get_particle_net(train_data, model_config)
model.summary()

2024-07-11 20:14:14.309875: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-07-11 20:14:14.326416: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-07-11 20:14:14.326502: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-07-11 20:14:14.331060: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-07-11 20:14:14.331124: I tensorflow/compile

Model: "ParticleNet"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 mask (InputLayer)           [(None, 500, 1)]             0         []                            
                                                                                                  
 MaskCoordShiftLayer (MaskC  (None, 500, 1)               0         ['mask[0][0]']                
 oordShiftLayer)                                                                                  
                                                                                                  
 points (InputLayer)         [(None, 500, 2)]             0         []                            
                                                                                                  
 AddShifttoCoordLayer_layer  (None, 500, 2)               0         ['MaskCoordShiftLaye

In [6]:
print(model(inputs))

KerasTensor(type_spec=TensorSpec(shape=(None, 2), dtype=tf.float32, name=None), name='ParticleNet/dense_2/Softmax:0', description="created by layer 'ParticleNet'")
